In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import yaml

In [6]:
folder_to_harvest = Path("../data/douhet")

json_paths = folder_to_harvest.glob("**/*.json")
json_paths_with_labels = zip(map(lambda x: x.stem.replace("_data", ""), json_paths), json_paths)


db = pd.DataFrame()

for data_label, data_path in json_paths_with_labels:

    course_label = data_label.split("_")[0]
    cours_edition_label = data_label.split("_")[1]

    associazione_path = Path(f"./data/{course_label}/{course_label}_{cours_edition_label}/associazione.xlsx")
    associazione_df = pd.read_excel(associazione_path)
    
    with open(data_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", data_label)
        course_df = course_df.merge(associazione_df, left_on="id", right_on="lettera").drop("lettera", axis=1)
        columns_to_reorder =  list(course_df.columns)
        course_df = course_df.loc[:, [ *columns_to_reorder[:1], columns_to_reorder[-1], *columns_to_reorder[1:-1] ]]
    
    db = pd.concat([db, course_df])

db.to_excel("data_abgrid_per_analisi.xlsx", index=False)

In [5]:
db.pivot_table(index="corso", values=["affiliation_coeff", "influence_coeff"], aggfunc="mean")

,affiliation_coeff,influence_coeff
corso,,
ursa_ii_stu,98.423477,99.231396
vega_i_stu,100.0,100.0
vega_ii_lud,100.0,100.0
zenit_i_stu,100.0,100.0
